In [1]:
import pandas as pd
from datetime import datetime, timedelta
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import re

In [2]:
def scroll_down(driver):
    # 페이지 맨 아래로 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)  # 스크롤 후 잠시 대기

def get_news_info(driver, query):
    # 기사 정보 추출
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    news_items = soup.find_all('div', class_='news_area')
    
    news_info = []
    for item in news_items:
        # 기사 제목 추출
        title_elem = item.find('a', class_='news_tit')
        title = title_elem.text.strip() if title_elem else None
        
        # 제목에 검색어가 포함되어 있지 않은 경우 크롤링하지 않음
        if not title or query not in title:
            continue

        # 기사 날짜 추출
        date_elems = item.find_all('span', class_='info')
        date = None
        for date_elem in date_elems:
            if date_elem.find('i', class_='spnew ico_paper'):
                continue  # i 태그가 있는 경우 건너뛰기
            # 날짜 형식에 맞는 텍스트 확인
            if re.match(r'\d{4}\.\d{2}\.\d{2}', date_elem.text.strip()):
                date = date_elem.text.strip()
                break
        
        if date:
            news_info.append({'제목': title, '날짜': date})
    
    return news_info

# 검색어 입력
query = input("뉴스 검색어를 입력하세요: ")

# 날짜 설정
start_date = datetime(2021, 6, 1)
end_date = datetime(2024, 5, 31)

# WebDriver로 Chrome 브라우저 열기
driver = webdriver.Chrome()

# 빈 리스트 생성
data = []

# 각 달마다 검색 수행
current_date = start_date
while current_date <= end_date:
    # 현재 달의 시작일과 종료일 계산
    next_month = current_date + timedelta(days=31)
    next_month = next_month.replace(day=1)  # 다음 달의 1일로 설정
    end_of_month = next_month - timedelta(days=1)  # 다음 달의 마지막 날짜로 설정
    
    # 네이버 뉴스 검색 페이지로 이동
    driver.get(f"https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={current_date.strftime('%Y.%m.%d')}&de={end_of_month.strftime('%Y.%m.%d')}")
    
    # 스크롤 다운을 100번 반복하여 데이터 가져오기 - 달 별로 1000개 
    for i in range(100): 
        scroll_down(driver)
        
    # 스크롤 다운이 완료된 후 데이터 가져오기
    news_info = get_news_info(driver, query)
    data.extend(news_info)
    
    # 다음 달로 이동
    current_date = next_month

# 브라우저 닫기
driver.quit()

# 데이터프레임 생성
df = pd.DataFrame(data)

# '제목' 열에서 중복 제거
df = df.drop_duplicates(subset='제목').reset_index(drop=True)

# 데이터프레임 출력
print(df)

                                                     제목           날짜
0                                  삼성생명, '뉴 올인원 암보험' 출시  2019.03.31.
1                                삼성생명 너마저… 시총 꺼지는 '보험주'  2019.03.30.
2                        윤석헌 금감원장, 삼성생명 즉시연금에 "가입했을수도…"  2019.03.27.
3                      금감원장 "종합검사 대상에 삼성생명 포함, 보복검사 없어"  2019.03.27.
4                                삼성생명 금감원 종합검사 올 하반기 가닥  2019.03.29.
...                                                 ...          ...
20153               삼성생명서비스노동조합 사무실 개소…문성현, 노사관계 청사진 제시  2024.01.09.
20154   [경기 전] 임근배 삼성생명 감독, "키아나 스미스가 선발로 나선다" … 구나단...  2024.01.15.
20155      [더벨][CFO Change]삼성카드 CFO에 삼성생명 출신 김태선 부사장 선임  2024.01.03.
20156   “자신감 많이 찾아”…항저우 AG 통해 부쩍 성장한 삼성생명 이해란, 2024년...  2024.01.01.
20157  [24W올스타전] 올스타전 첫 출전한 삼성생명 이주연, "(임근배) 감독님과 함께...  2024.01.06.

[20158 rows x 2 columns]


In [3]:
df.to_csv('./baf/삼성생명.csv', index=False)